# Precipitation Variability Across Timescales

This notebook demonstrates how to use the precipitation variability metrics driver. 

This notebook should be run in an environment with python, jupyterlab, pcmdi metrics package, and cdat installed. It is expected that you have downloaded the sample data as demonstrated in [the download notebook](Demo_0_download_data.ipynb).  

The following cell reads in the choices you made during the download data step:

In [2]:
from user_choices import demo_data_directory, demo_output_directory

## Basic Use

Use the `--help` flag for assistance with the precip variability driver:

In [5]:
%%bash
variability_across_timescales_PS_driver.py --help

Deprecation warning: please use 'import pcmdi_metrics.driver.pmp_parser.PMPParser'
usage: variability_across_timescales_PS_driver.py [-h]
                                                  [--parameters PARAMETERS]
                                                  [--diags OTHER_PARAMETERS [OTHER_PARAMETERS ...]]
                                                  [--mip MIP] [--mod MOD]
                                                  [--var VAR] [--frq FRQ]
                                                  [--modpath MODPATH]
                                                  [--outdir OUTDIR]
                                                  [--prd PRD] [--fac FAC]
                                                  [--nperseg NPERSEG]
                                                  [--noverlap NOVERLAP]

optional arguments:
  -h, --help            show this help message and exit
  --parameters PARAMETERS, -p PARAMETERS
  --diags OTHER_PARAMETERS [OTHER_PARAMETERS ...], -d OTHER_PARAME

Settings are specified in a separate parameter file, which is printed here:

In [6]:
# print parameter file
with open("basic_precip_variability_param.py") as f:
    print(f.read())

mip = "cmip5"
exp = "historical"
mod = "GISS-E2-H"
var = "pr"
frq = "day"
modpath = 'demo_data_standard/CMIP5_demo_timeseries/historical/atmos/day/pr/pr_day_GISS-E2-H_historical_r6i1p1_20000101-20051231.nc'
outdir = 'demo_output/precip_variability'
prd = [2000,2005]  # analysis period
fac = 86400  # factor to make unit of [mm/day]

# length of segment in power spectra (~10 years)
# shortened to 2 years for demo purposes
nperseg = 2 * 365 * 8  
# length of overlap between segments in power spectra (~5 years)
# shortened to 1 year for demo purposes
noverlap = 1 * 365 * 8



The parameter file is passed to the driver using the `-p` flag, similar to other PMP metrics. The next cell runs the driver as a subprocess:

In [1]:
%%bash
variability_across_timescales_PS_driver.py -p basic_precip_variability_param.py

Deprecation warning: please use 'import pcmdi_metrics.driver.pmp_parser.PMPParser'
demo_data_standard/CMIP5_demo_timeseries/historical/atmos/day/pr/pr_day_GISS-E2-H_historical_r6i1p1_20000101-20051231.nc
GISS-E2-H
[2000, 2001]
730 365
# of data: 0
[]


## Power Spectrum Parameters

Two parameters depend on the length of the analysis period and data frequency. They need to specified by the user.

**nperseg**  
The parameter `nperseg` is the length of a segment in the power spectra. 10 years is a good value to pick. 

The general formula for `nperseg` is:  
`nperseg = #years * 365 * 24/(timestep in hours)`

For daily data this would look like:  
`nperseg = 10 * 365`  

For 3-hourly data this need to be adjusted by a factor of 8 (8 time steps in a 24-hour day):  
`nperseg = 10 * 365 * 8`

**noverlap**  
The parameter `noverlap` is the length of overlap between segments in the power spectra. 5 years is a good value to pick.

Similar to `nperseg`, the formula for `noverlap`:  
`noverlap = #years * 365 * 24/(timestep in hours)` 

For daily data this would look like:  
`noverlap = 5 * 365`  

For 3-hourly data this would look like:  
`noverlap = 5 * 365 * 8`

With this in mind, we run the precip variability driver on a short daily observational timeseries.

In [2]:
%%bash
variability_across_timescales_PS_driver.py -p basic_precip_variability_param.py \
--modpath demo_data_standard/PCMDIobs2/atmos/day/pr/GPCP-IP/gn/v20200719/pr.day.GPCP-IP.BE.gn.v20200719.1998-1999.xml \
--mod 'GPCP-IP' \
--prd [1998,1999] \
--nperseg 730 \
--noverlap 365

Deprecation warning: please use 'import pcmdi_metrics.driver.pmp_parser.PMPParser'
demo_data_standard/PCMDIobs2/atmos/day/pr/GPCP-IP/gn/v20200719/pr.day.GPCP-IP.BE.gn.v20200719.1998-1999.xml
GPCP-IP
[1998,1999]
730 365
# of data: 0
[]
